D'abord : se logguer à un compte gmail \
login : eleveenmi@gmail.com  (i=1,2,3,..) \
mwd : pwdeleveEnmi (i=1,2,3,..)

Avant de lancer le notebook, il faut avoir déplace le fichier learning.tar.gz du dossier partagé dans le dossier `stage_secondes_2025`.

#0) Installations, imports et fonctions

In [1]:
# Commencez par installer et faire les imports pour
 # avoir toutes les fonctions utiles
!pip install -q plotly
import os
import tarfile
import shutil
from google.colab import drive
from PIL import Image, ImageOps
import random
import torchvision.transforms as T
from matplotlib import pyplot as plt
import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, dataset
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import confusion_matrix
# Spécifique au TP :
! git clone https://github.com/nanopiero/intro-ia-secondes-2025.git
! cp intro-ia-secondes-2025/utils/apprentissage.py .
# from google.colab import files
# uploaded = files.upload()
from importlib import reload
import apprentissage
# reload(apprentissage)

from apprentissage import (
    show_confusion_matrix_plotly,
    get_valid_classes,
    clean_directory,
    apply_transforms,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    RandomDistortion,
    RandomCropping,
    Rotation,
    RandomColorJitter,
    show_filters,
    show_class_prediction_gallery_by_name,
    find_file_id
)


Cloning into 'intro-ia-secondes-2025'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 239 (delta 67), reused 8 (delta 8), pack-reused 135 (from 1)
Receiving objects: 100% (239/239), 7.98 MiB | 26.11 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [ ]:
drive.mount('/content/drive')

In [ ]:
 # Commencez par installer et faire les imports pour
 # avoir toutes les fonctions utiles
!pip install -q plotly
import os
import tarfile
import shutil
from google.colab import drive
from PIL import Image, ImageOps
import random
import torchvision.transforms as T
from matplotlib import pyplot as plt
import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, dataset
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import confusion_matrix
# Spécifique au TP :
! git clone https://github.com/nanopiero/intro-ia-secondes-2025.git
! cp intro-ia-secondes-2025/utils/annotation.py .
! cp intro-ia-secondes-2025/utils/configuration.py .
# from google.colab import files
# uploaded = files.upload()
from importlib import reload
import configuration as cf
import annotation as an
# reload(cf)
# reload(an)

def show_confusion_matrix_plotly(model, val_loader, class_names):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            outputs = model(x)
            preds.extend(outputs.argmax(1).cpu().numpy())
            labels.extend(y.numpy())

    cm = confusion_matrix(labels, preds)
    df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)

    fig = px.imshow(
        df_cm,
        text_auto=True,
        color_continuous_scale='Blues',
        labels=dict(x="Predicted", y="True", color="Count"),
        title="Confusion Matrix (Validation Set)",
        width=800,
        height=800
    )
    fig.update_layout(font=dict(size=14))
    fig.show()

def get_valid_classes(train_dir, val_dir, min_count=5):
    train_classes = set(os.listdir(train_dir))
    val_classes = set(os.listdir(val_dir))
    common_classes = train_classes & val_classes
    valid_classes = []

    for cls in common_classes:
        train_cls_path = os.path.join(train_dir, cls)
        val_cls_path = os.path.join(val_dir, cls)

        if not os.path.isdir(train_cls_path) or not os.path.isdir(val_cls_path):
            continue

        train_count = len([f for f in os.listdir(train_cls_path) if os.path.isfile(os.path.join(train_cls_path, f))])
        val_count = len([f for f in os.listdir(val_cls_path) if os.path.isfile(os.path.join(val_cls_path, f))])

        if train_count >= min_count and val_count >= min_count:
            valid_classes.append(cls)

    return set(valid_classes)
def clean_directory(dir_path, valid_classes):
    for cls in os.listdir(dir_path):
        full_path = os.path.join(dir_path, cls)
        if os.path.isdir(full_path) and cls not in valid_classes:
            shutil.rmtree(full_path)


# Apply each transform in the list sequentially
def apply_transforms(img, transform_list):
    for t in transform_list:
        img = t(img)
    return img

class RandomHorizontalFlip:
    def __call__(self, img):
        if random.random() > 0.5:
            return ImageOps.mirror(img)  # left ↔ right
        return img

class RandomVerticalFlip:
    def __call__(self, img):
        if random.random() > 0.5:
            return ImageOps.flip(img)  # top ↕ bottom
        return img

class RandomDistortion:
    def __init__(self, intensity=5):
        self.intensity = intensity

    def __call__(self, img):
        w, h = img.size
        dx = self.intensity
        dy = self.intensity
        coeffs = (
            1, random.uniform(-dx/h, dx/h), 0,
            random.uniform(-dy/w, dy/w), 1, 0
        )
        return img.transform(img.size, Image.AFFINE, coeffs)

class RandomCropping:
    def __init__(self, zoommax=0.1):
        self.zoommax = zoommax

    def __call__(self, img):
        w, h = img.size
        dw = int(random.uniform(0, self.zoommax) * w)
        dh = int(random.uniform(0, self.zoommax) * h)
        left = dw
        top = dh
        right = w - dw
        bottom = h - dh
        return img.crop((left, top, right, bottom)).resize((w, h))

class Rotation:
    def __init__(self, degreemax=10):
        self.degreemax = degreemax

    def __call__(self, img):
        angle = random.uniform(-self.degreemax, self.degreemax)
        return img.rotate(angle)

class RandomColorJitter:
    def __init__(self, contrast=0.2, hue=0.1, saturation=0.2, brightness=0.2):
        self.transform = T.ColorJitter(
            contrast=contrast,
            hue=hue,
            saturation=saturation,
            brightness=brightness
        )

    def __call__(self, img):
        return self.transform(img)


def show_filters(weights, title):
    fig, axes = plt.subplots(8, 8, figsize=(10, 10))
    fig.suptitle(title, fontsize=16)
    for i, ax in enumerate(axes.flat):
        filt = weights[i, 0, :, :].cpu().numpy()  # R channel
        ax.imshow(filt, cmap='bwr', vmin=-0.2, vmax=0.2)
        ax.axis('off')
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)
    plt.show()


def show_class_prediction_gallery_by_name(model, val_loader, class_names, class_name, max_per_col=10):
    assert class_name in class_names, f"'{class_name}' not found in class names: {class_names}"
    K = class_names.index(class_name)

    model.eval()
    all_images, all_labels, all_preds = [], [], []

    # Gather predictions and labels
    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            preds = model(x).argmax(1).cpu()
            all_images.extend(x.cpu())
            all_labels.extend(y.cpu())
            all_preds.extend(preds.cpu())

    # Categorize samples
    correct, missed, false_alarm = [], [], []
    for img, label, pred in zip(all_images, all_labels, all_preds):
        if label == K and pred == K:
            correct.append((img, label, pred))
        elif label == K and pred != K:
            missed.append((img, label, pred))
        elif label != K and pred == K:
            false_alarm.append((img, label, pred))

    def sample(lst): return random.sample(lst, min(len(lst), max_per_col))
    correct_sample = sample(correct)
    missed_sample = sample(missed)
    false_alarm_sample = sample(false_alarm)

    # Plot
    fig, axes = plt.subplots(max_per_col, 3, figsize=(12, 2.5 * max_per_col))
    fig.suptitle(f'Class "{class_name}" (index {K}) — Prediction Gallery', fontsize=16)

    for i in range(max_per_col):
        for j, sample_list in enumerate([correct_sample, missed_sample, false_alarm_sample]):
            ax = axes[i, j] if max_per_col > 1 else axes[j]
            if i < len(sample_list):
                img, label, pred = sample_list[i]
                ax.imshow(np.transpose(img.numpy(), (1, 2, 0)))
                ax.set_title(f"True: {class_names[label]}\nPred: {class_names[pred]}")
            ax.axis("off")

    col_titles = ["Correct (TP)", "Missed (FN)", "False Alarm (FP)"]
    for j in range(3):
        axes[0, j].set_title(col_titles[j], fontsize=14)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

def find_file_id(filename):
    query = f"name='{filename}' and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get("files", [])
    if not files:
        raise FileNotFoundError(f"{filename} not found in Drive.")
    return files[0]['id']



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# I) Chargeons le jeu d'entraînement

In [2]:
# D'abord, chargeons les images
tar_path = '/content/drive/MyDrive/stage_secondes_2025/learning.tar.gz'
extract_path = '/content/'

with tarfile.open(tar_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

# Ensuite précisons où sont les deux dossiers pour l'entraînelment et la validation
base_dir = os.path.join(extract_path, 'learning')
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/stage_secondes_2025/learning.tar.gz'

In [ ]:
# Si dossiers vides :
# Step 3: Check for common classes and filter those with >=10 elements in both
valid_classes = get_valid_classes(train_dir, val_dir)

# Step 4: Remove invalid class-subfolders
clean_directory(train_dir, valid_classes)
clean_directory(val_dir, valid_classes)

print("Finished cleaning. Valid classes kept:", sorted(valid_classes))

#II) Choix d'une augmentation de données

In [ ]:
# Load one image to test
image_path = "learning/train/21/" + os.listdir("learning/train/21")[0]
img = Image.open(image_path).convert("RGB")

# Define your test transform list
transforms = [
    RandomVerticalFlip(),
    RandomCropping(0.1),
    Rotation(10),
    RandomColorJitter(contrast=0.3, hue=0.05, saturation=0.3, brightness=0.3)
]



# Show the original and augmented images
aug_img = apply_transforms(img.copy(), transforms)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original")
plt.imshow(img)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Augmented")
plt.imshow(aug_img)
plt.axis("off")
plt.show()

#III) Entraînement d'un réseau de neurones

##III)1) Qu'est-ce qu'il y a dans un réseau de neurone (pour le traitement d'image) ?

In [ ]:
import torch
import torchvision.models as models
import matplotlib.pyplot as plt

# Load pretrained and randomly initialized ResNet50
resnet_pretrained = models.resnet50(pretrained=True)
resnet_random = models.resnet50(pretrained=False)

# Get first conv layer weights: shape = [64, 3, 7, 7]
w_pre = resnet_pretrained.conv1.weight.data.clone()
w_rand = resnet_random.conv1.weight.data.clone()

show_filters(w_rand, "Random Init - First Conv Layer (R channel)")
show_filters(w_pre, "Pretrained on ImageNet - First Conv Layer (R channel)")
del resnet_pretrained, resnet_random

https://github.com/vdumoulin/conv_arithmetic

https://www.image-net.org/challenges/LSVRC/2012/index#task

## III)2) Comment entraîner un tel réseau ?

In [ ]:
# ==== CONFIGURATION  ====
config = {
    'architecture': 'resnet18',   # 'resnet18' or 'resnet50'
    'freeze_backbone': False,
    'learning_rate': 0.001,
    'batch_size': 32,
    'num_epochs': 5,
    'image_size': 224,
    'train_dir': 'learning/train',
    'val_dir': 'learning/val',

    'transforms': [
        RandomVerticalFlip(),
        RandomCropping(0.1),
        Rotation(10),
        RandomColorJitter(contrast=0.3, hue=0.05, saturation=0.3, brightness=0.3)
    ]
}

In [ ]:

# Compose training transform from config list
train_transform = T.Compose(config['transforms'] + [
    T.Resize((config['image_size'], config['image_size'])),
    T.ToTensor()
])

val_transform = T.Compose([
    T.Resize((config['image_size'], config['image_size'])),
    T.ToTensor()
])



train_set = ImageFolder(config['train_dir'], transform=train_transform)
val_set = ImageFolder(config['val_dir'], transform=val_transform)
class_names = train_set.classes

train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_set, batch_size=config['batch_size'], shuffle=False)


# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model architecture
if config['architecture'] == 'resnet18':
    model = models.resnet18(pretrained=True)
elif config['architecture'] == 'resnet50':
    model = models.resnet50(pretrained=True)
else:
    raise ValueError("Unsupported architecture.")

# Freeze backbone if needed
if config['freeze_backbone']:
    for param in model.parameters():
        param.requires_grad = False

# Replace classifier
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

# Metric tracking
train_accs, val_accs = [], []
train_bal_accs, val_bal_accs = [], []

# === Training Loop ===
for epoch in range(config['num_epochs']):
    model.train()
    train_preds, train_labels = [], []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(train_labels, train_preds)
    train_bal = balanced_accuracy_score(train_labels, train_preds)
    train_accs.append(train_acc)
    train_bal_accs.append(train_bal)

    # Validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    val_bal = balanced_accuracy_score(val_labels, val_preds)
    val_accs.append(val_acc)
    val_bal_accs.append(val_bal)

    print(f"Epoch {epoch+1}/{config['num_epochs']}: "
          f"Train Acc: {train_acc:.3f}, Val Acc: {val_acc:.3f} | "
          f"Train BalAcc: {train_bal:.3f}, Val BalAcc: {val_bal:.3f}")


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Val Accuracy')
plt.plot(train_bal_accs, label='Train Balanced Acc', linestyle='--')
plt.plot(val_bal_accs, label='Val Balanced Acc', linestyle='--')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training & Validation Metrics')
plt.legend()
plt.grid(True)
plt.show()

#IV) Visualisation des prédictions sur le jeu de validation

In [ ]:
show_confusion_matrix_plotly(model, val_loader, class_names, device)


In [ ]:
show_class_prediction_gallery_by_name(model, val_loader, class_names, device, class_name='20')

#V) Sauvegarde du réseau et partage vers le compte central

In [ ]:
groupe = 14

In [ ]:
# Save model weights
arch = config['architecture']
imgsize = config['image_size']
filename = f"{groupe}_{arch}_{imgsize}.pth"
torch.save(model.state_dict(), filename)

from google.colab import drive
import shutil
import os

# Mount drive if not already mounted
drive.mount('/content/drive')

drive_dir = "/content/drive/MyDrive/stage_secondes_2025"
os.makedirs(drive_dir, exist_ok=True)

# Move file
dst_path = os.path.join(drive_dir, filename)
shutil.move(filename, dst_path)
print(f"Saved to {dst_path}")


from google.colab import auth
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')

# Find file ID by name in destination folder
file_id = find_file_id(filename, drive_service)

# Share with email
drive_service.permissions().create(
    fileId=file_id,
    body={
        'type': 'user',
        'role': 'reader',
        'emailAddress': 'tpdeeplearning@gmail.com'
    },
    fields='id'
).execute()

print(f"Shared {filename} with tpdeeplearning@gmail.com")